# Path of destruction

Python script to import Hurricane data and a heuristical model for estimating home damage. We use data for Hurricane Irma from last year in our case study.

---

### Hurricane Irma 
September 8, 2017, 12 Zulu, Created @ ~9AM EST data forecasts

Source: https://www.nhc.noaa.gov/gis/archive_forecast_results.php?id=al11&year=2017&name=Hurricane%20IRMA  
File1: al112017_5day_037.zip (8:59 AM)    
File2: al112017_fcst_037.zip (9:00 AM)

In [1]:
# import packages
import pandas as pd 
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import geojson
from geojson import Point, Feature, FeatureCollection, dump
from shapely.geometry import shape, Point
from geopy.distance import geodesic
import tempfile
import fiona

import warnings
warnings.filterwarnings('ignore')

Import data from NOAA

In [2]:
HURRICANE_PATH = './al112017_5day_037/al112017-037_5day_lin.shp'
HURRICANE_ERROR_CONE = './al112017_5day_037/al112017-037_5day_pgn.shp'
HURRICANE_WIND_RADII = './al112017_fcst_037/al112017_2017090809_forecastradii.shp'

In [3]:
# path of hurricane
path = fiona.open(HURRICANE_PATH)
first_path = path.next()

path_features = []
# entire path of hurricane
path_features.append(Feature(geometry = first_path['geometry'], properties = {"hurricane": "Irma"}))

# add more features...
for i in range(0, 6):
    point = Point(first_path['geometry']['coordinates'][i])
    day = str(datetime.date(2008, 9, 10) + datetime.timedelta(i))
    # 5d predictions
    path_features.append(Feature(geometry = point, properties = {"day": day}))

In [4]:
# hurricane cone
cone = fiona.open(HURRICANE_ERROR_CONE)
first_cone = cone.next()
cone_features = []
cone_features.append(Feature(geometry = first_cone['geometry'], properties={"shape": "Prediction"})) 

In [42]:
# hurricane wind radii
radius = fiona.open(HURRICANE_WIND_RADII)
radius_features = []
for i in range(len(radius)):
    rad_num = radius[i]
    radius_features.append(Feature(geometry = rad_num['geometry'], properties = {"shape": "Prediction"})) 

## TODO: change by color

# process radii data - 3, 3, 3, 3, 2, 2 of radii
def apply_radius_list(radius_features):
    radius_df = {}
    for i in [0, 1, 2, 3]:
        radius_df[i] = [radius_features[j] for j in range(i, i + 3)]
    for i in [4, 5]:
        radius_df[i] = [radius_features[j] for j in range(i, i + 2)]
#         ['geometry']['coordinates'][0]
    return radius_df

radius_data = apply_radius_list(radius_features)

In [6]:
# dump data for Kepler
def generate_kepler_data():
    with open('./geojson/path.geojson', 'w') as f:
        dump(FeatureCollection(path_features), f)
    
    with open('./geojson/cone.geojson', 'w') as f:
        dump(FeatureCollection(cone_features), f)

    with open('./geojson/radii.geojson', 'w') as f:
        dump(FeatureCollection(radius_features), f)
        
generate_kepler_data()

Input data class and Point initialization for latitude longitude. Let's use Key West.

In [7]:
KeyWest = Point((-81.47, 24.33))
KeyWest = Point((-75.34144, 22.12434))
day = datetime.date(2018, 9, 12)
day_num = (day - datetime.date(2018, 9, 10)).days
day_num

2

In [44]:
shape(radius_features[12].geometry).contains(Point((-79.94144, 24.82434)))

True

In [51]:
funcer = 

In [53]:
# in radius
def find_in_radius(point, day_num = day_num):
    found = []
    radius_subset = radius_data[day_num]
    counter = 0
    for i in funcer[day_num]:
        counter = counter + 1
        polygon = shape(radius_features[i].geometry)
        if(polygon.contains(point)):
            point_num = 1 if (i == 0) else (-1 if (i == len(radius_subset) - 1) else 0)
            point_name = 'fastest' if (i == 0) else ('slowest' if (i == len(radius_subset) - 1) else 'middle')
            print('Found your location on day {} at {} wind radius'.format(day_num, point_name))
            found.append(point_num) # days separate
    return found

# Missing days?
find_in_radius(Point((-79.94144, 24.82434)), 5)

Found your location on day 5 at middle wind radius


[0]

In [9]:
# # find distances to each circle
# def find_distance_circle(point = KeyWest, day_num = day_num):
#     point_coord = (point.coords.xy[0][0], point.coords.xy[1][0])
#     found = {}
#     radius_subset = radius_data[day_num]
#     for i in range(len(radius_subset)):
#         polygon = shape(radius_subset[i].geometry)
#         point_num = 1 if (i == 0) else (-1 if (i == len(radius_subset) - 1) else 0)

#         if(polygon.contains(point)):
#             found[point_num] = '' # don't include ones we're in
#         else:
#             found[point_num] = min([geodesic(point_coord, j).miles for j in polygon.exterior.coords]) # min distance
#     return found

# find_distance_circle(day_num = 2)

In [10]:
# # find distances to the eye
# def find_distance_eye(point = KeyWest, day_num = day_num):
#     point_coord = (point.coords.xy[0][0], point.coords.xy[1][0])
#     return geodesic(point_coord, tuple(path_features[day_num + 1]['geometry']['coordinates'])).miles

# find_distance_eye()

# model specification
- is it contained in the first radius
- if not, second radius?
- if not, 3rd radius?
- nearest radius it is not contained to's distance
- distance from the eye

In [11]:
day_num = 4

In [12]:
# # nearest radius
# # radius_found = [1, 0, -1]
# # at most 0.4
# dist = find_distance_circle(KeyWest, day_num)
# try:
#     if (max(radius_found) == 1):
#         radii_distance_weight = 0.4
#     else:
#         radii_distance_weight = (1 - dist[max(radius_found) + 1]/100) * (0.3 + 0.1 * max(radius_found))
# except: # none were found - outside all of the radii
#     radii_distance_weight = (1 - dist[max(radius_found) + 1]/100) * 0.1
# radii_distance_weight

In [13]:
# distance_weight = (1 - find_distance_eye(KeyWest, day_num)/1000) * 0.25
# distance_weight

In [14]:
# # total 
# distance_weight + radii_distance_weight + radius_weight

In [15]:
# class structure
class Insuricane():
    
    ### Initialization
    def __init__(self, Point, 
                 base_date = datetime.date(2017, 9, 8),
                 path_features = path_features, 
                 cone_features = cone_features, 
                 radius_data = radius_data):
        self.date = base_date # base date
        self.point = Point
        self.path_features = path_features
        self.cone_features = cone_features
        self.radius_data = radius_data
        
    def get_day_num(self, day):
        return min(5, (day - self.date).days)

    ### Calculating locations with our geojson data
    # which radii is it in for wind?
    def find_in_radius(self, day, suppressPrint = False):
        found = []
        radius_subset = self.radius_data[self.get_day_num(day)]
        for i in range(len(radius_subset)):
            polygon = shape(radius_subset[i].geometry)
            if(polygon.contains(point)):
                point_num = 1 if (i == 0) else (-1 if (i == len(radius_subset) - 1) else 0)
                point_name = 'fastest' if (i == 0) else ('slowest' if (i == len(radius_subset) - 1) else 'middle')
                
                if (suppressPrint == False):
                    print('Found your location on day {} at {} wind radius'.format(
                        self.get_day_num(day), point_name))
                found.append(point_num) # days separate
        return found
    
    # closest radii
    def find_distance_circle(self, day):
        point_coord = (self.point.coords.xy[0][0], self.point.coords.xy[1][0])
        found = {}
        radius_subset = self.radius_data[self.get_day_num(day)]
        for i in range(len(radius_subset)):
            polygon = shape(radius_subset[i].geometry)
            point_num = 1 if (i == 0) else (-1 if (i == len(radius_subset) - 1) else 0)

            if(polygon.contains(point)):
                found[point_num] = '' # don't include ones we're in
            else:
                found[point_num] = min([geodesic(point_coord, j).miles for j in polygon.exterior.coords])
        return found
    
    # distance to the eye on a given day
    def find_distance_eye(self, day):
        point_coord = (self.point.coords.xy[0][0], self.point.coords.xy[1][0])
        return geodesic(point_coord, 
                        tuple(path_features[self.get_day_num(day) + 1]['geometry']['coordinates'])).miles
    
    
    ### We want this to be able to change per day. We have the eye by day, we have radii by day.
    # radius weighting
    def __calc_radius_weight(self, day, **kwargs):
        radius_found = self.find_in_radius(day, **kwargs)
        try:
            radius_weight = 0.25 + float(max(radius_found) * 0.1) # 0.35, 0.25, 0.15
        except: # none were found - outside all of the radii
            radius_weight = 0
        return radius_weight

    # loop weighting
    def __calc_circle_weight(self, day, **kwargs):
        radius_found = self.find_in_radius(day, **kwargs)
        dist = self.find_distance_circle(day)
        try:
            if (max(radius_found) == 1):
                radii_distance_weight = 0.4
            else:
                radii_distance_weight = (1 - dist[max(radius_found) + 1]/100) * (0.3 + 0.1 * max(radius_found))
        except: # none were found - outside all of the radii
            radii_distance_weight = (1 - dist[max(radius_found) + 1]/100) * 0.1
        return radii_distance_weight

    ### Actual value 'heuristics' model
    # chance your house is destroyed
    def __calc_chance_hurricane(self, day, **kwargs):
        distance_weight = (1 - self.find_distance_eye(day)/1000) * 0.25
        return distance_weight + self.__calc_radius_weight(day, **kwargs) + self.__calc_circle_weight(day, **kwargs)
    
    # does this for each of the 5 days
    def calc_risk(self, **kwargs):
        hurrichance = {}
        for i in range(7):
            second_day = self.date + datetime.timedelta(i) # increment days
            hurrichance[second_day] = self.__calc_chance_hurricane(second_day, **kwargs)
        return pd.DataFrame(hurrichance, index = ['chance']).T
    
    # maximum chance of a hurricane
    def calc_max_hurricane(self):
        hurricane_a1 = self.calc_risk(suppressPrint = True) # never print for this
        hurricane_a1 = hurricane_a1[hurricane_a1.index != min(hurricane_a1.index)]
        hurricane_temp = hurricane_a1[hurricane_a1['chance'] == max(hurricane_a1['chance'])]
        return (hurricane_temp['chance'])

In [16]:
# initialize
BCG = Insuricane(KeyWest)

In [17]:
radius_data[5][0]

{"geometry": {"coordinates": [[[-75.689735, 23.18136], [-75.669212, 23.181028], [-75.648697, 23.180365], [-75.628189, 23.179369], [-75.607712, 23.178043], [-75.587265, 23.176384], [-75.566849, 23.174397], [-75.546478, 23.172079], [-75.526146, 23.169432], [-75.505875, 23.166456], [-75.485664, 23.163153], [-75.465523, 23.159523], [-75.445457, 23.155569], [-75.425461, 23.151293], [-75.405556, 23.146692], [-75.385742, 23.141769], [-75.366028, 23.136528], [-75.346413, 23.130968], [-75.326912, 23.125093], [-75.307526, 23.118902], [-75.288261, 23.112398], [-75.269127, 23.105587], [-75.25013, 23.098465], [-75.231262, 23.091038], [-75.212547, 23.083307], [-75.193985, 23.075274], [-75.175583, 23.066942], [-75.157341, 23.058313], [-75.139267, 23.049393], [-75.121361, 23.04018], [-75.103645, 23.03068], [-75.086105, 23.020895], [-75.068764, 23.010826], [-75.05162, 23.000481], [-75.034668, 22.989857], [-75.017937, 22.978964], [-75.001404, 22.967798], [-74.985092, 22.956371], [-74.969002, 22.944679],

In [18]:
# chance of hurricane
BCG.calc_risk(suppressPrint = True)

ValueError: max() arg is an empty sequence

In [ ]:
# most dangerous day
BCG.calc_max_hurricane()

In [ ]:
# plot chance of getting owned by Irma
def plot_irmachance(city = 'Key West', point = KeyWest, alpha = 0.5):
    BCG = Insuricane(point)
    hurricane_ts = BCG.calc_risk(suppressPrint = True)
    
    hurricane_df = hurricane_ts[datetime.date(2017, 9, 9):]
    hurricane_df['ewm'] = hurricane_df.ewm(alpha).mean()
    hurricane_df = hurricane_df.applymap(lambda x: 100 * x)

    sns.set(font_scale = 1.1)
    sns.set_style('whitegrid')
    plt.figure(figsize = (8, 6))
    plt.plot(hurricane_df['chance'], marker = 'o', color = '#4285f4', alpha = 0.75, label = 'Chance of hurricane')
    plt.plot(hurricane_df['ewm'], linestyle = '--', color = '#DC143C', alpha = 0.5, label = 'Rolling likelihood')
    plt.xticks(rotation = -15)
    plt.xlabel('Date')
    plt.ylabel('Chance of hurricane')
    plt.title('Chance of {} homes destroyed'.format(city))
    
    plt.plot(BCG.calc_max_hurricane().index.values[0], 
             BCG.calc_max_hurricane().values[0] * 100, 
             color = '#4285f4', marker = 's', linestyle = 'none', label='Riskiest day')
    
    plt.legend()
    plt.show()

In [ ]:
plot_irmachance()